In [21]:
function frag_face(V, EV, FE, sp_idx, sigma)

    vs_num = size(V, 1)
    

	# 2D transformation of sigma face
    sigmavs = (abs.(FE[sigma:sigma,:])*abs.(EV))[1,:].nzind
    
    sV = V[sigmavs, :]
    sEV = EV[FE[sigma, :].nzind, sigmavs]
    M = Lar.Arrangement.submanifold_mapping(sV)
    tV = ([V ones(vs_num)]*M)[:, 1:3]  # folle convertire *tutti* i vertici
    sV = tV[sigmavs, :]
    # sigma face intersection with faces in sp_idx[sigma]
    for i in sp_idx[sigma]
        @show i
        tmpV, tmpEV = Lar.Arrangement.face_int(tV, EV, FE[i, :])
       

        
        sV, sEV = Lar.skel_merge(sV, sEV, tmpV, tmpEV)
    end

    # computation of 2D arrangement of sigma face
    sV = sV[:, 1:2]
    nV, nEV, nFE = Lar.Arrangement.planar_arrangement(sV, sEV, sparsevec(ones(Int8, length(sigmavs))))
    if nV == nothing ## not possible !! ... (each original face maps to its decomposition)
        return [], spzeros(Int8, 0,0), spzeros(Int8, 0,0)
    end
    nvsize = size(nV, 1)
    nV = [nV zeros(nvsize) ones(nvsize)]*inv(M)[:, 1:3] ## ????
    
    return nV, nEV, nFE
end

frag_face (generic function with 1 method)

In [22]:
using SparseArrays
using LinearAlgebraicRepresentation

Lar=LinearAlgebraicRepresentation



V = Float64[
        0 0 0; 0 1 0;
        1 1 0; 1 0 0;
        0 0 1; 0 1 1;
        1 1 1; 1 0 1
    ]

    EV = sparse(Int8[
        -1  1  0  0  0  0  0  0;
        0 -1  1  0  0  0  0  0;
        0  0 -1  1  0  0  0  0;
        -1  0  0  1  0  0  0  0;
        -1  0  0  0  1  0  0  0;
        0 -1  0  0  0  1  0  0;
        0  0 -1  0  0  0  1  0;
        0  0  0 -1  0  0  0  1;
        0  0  0  0 -1  1  0  0;
        0  0  0  0  0 -1  1  0;
        0  0  0  0  0  0 -1  1;
        0  0  0  0 -1  0  0  1;
    ])

    FE = sparse(Int8[
        1  1  1 -1  0  0  0  0  0  0  0  0;
        0  0  0  0  0  0  0  0 -1 -1 -1  1;
        -1  0  0  0  1 -1  0  0  1  0  0  0;
        0 -1  0  0  0  1 -1  0  0  1  0  0;
        0  0 -1  0  0  0  1 -1  0  0  1  0;
        0  0  0  1 -1  0  0  1  0  0  0 -1;
    ])

@time frag_face(Lar.Points(V),EV,FE,[2,3,4,5],2)


i = 3
  0.044958 seconds (58.92 k allocations: 3.000 MiB)


([0.0 0.0 1.0; 0.0 1.0 1.0; 1.0 1.0 1.0; 1.0 0.0 1.0], 
  [1, 1]  =  -1
  [4, 1]  =  -1
  [1, 2]  =  1
  [2, 2]  =  -1
  [2, 3]  =  1
  [3, 3]  =  -1
  [3, 4]  =  1
  [4, 4]  =  1, 
  [1, 1]  =  1
  [1, 2]  =  1
  [1, 3]  =  1
  [1, 4]  =  -1)

In [23]:
using BenchmarkTools
@btime frag_face(Lar.Points(V),EV,FE,[2,3,4,5],2)

i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 

i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 

i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 

i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 

i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 

i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 3
i = 

([0.0 0.0 1.0; 0.0 1.0 1.0; 1.0 1.0 1.0; 1.0 0.0 1.0], 
  [1, 1]  =  -1
  [4, 1]  =  -1
  [1, 2]  =  1
  [2, 2]  =  -1
  [2, 3]  =  1
  [3, 3]  =  -1
  [3, 4]  =  1
  [4, 4]  =  1, 
  [1, 1]  =  1
  [1, 2]  =  1
  [1, 3]  =  1
  [1, 4]  =  -1)

In [145]:

#function duetrasf(V,EV,FE,sigma,vs_num)
function duetrasf(V::Lar.Points,EV::Lar.ChainOp,FE::Lar.ChainOp,sigma::Int,vs_num::Int) 
  
    sigmavs = (abs.(FE[sigma:sigma,:])*abs.(EV))[1,:].nzind
    sV = V[sigmavs, :]
    sEV = EV[FE[sigma, :].nzind, sigmavs]
    #non si può dividire la funzione perchè submanifold_mapping richiede un'array/matrice lungo tot 
    M = Lar.Arrangement.submanifold_mapping(sV)
    
       
            
    tV = ([V ones(vs_num)]*M)[:, 1:3]  # folle convertire *tutti* i vertici
    sV = tV[sigmavs, :]
    
    return tV, sV, sEV, M, sigmavs

    end

duetrasf (generic function with 1 method)

In [146]:
#Tuple{Array{Float64,2},SparseMatrixCSC{Int8,Int64},SparseMatrixCSC{Int8,Int64},Array{Any,2},SparseMatrixCSC{Int8,Int64},Array{Int64,1},Int64
#function sigma(tV,EV,FE,sV,sEV,sp_idx,sigma)
#function sigma(tV::Array,EV::Lar.ChainOp,FE::Lar.ChainOp,sV::Array,sEV::Lar.ChainOp,sp_idx::Array,sigma::Int)
function sigma(tV::Tuple{Array{Float64,2}},EV::SparseMatrixCSC{Int8,Int64},FE::SparseMatrixCSC{Int8,Int64},sV::Array{Any,2},sEV::SparseMatrixCSC{Int8,Int64},sp_idx::Array{Int64,1},sigma::Int64,)

    
    for i in sp_idx[sigma]
        tmpV, tmpEV = Lar.Arrangement.face_int(tV, EV, FE[i, :])
        #@show FE[i, :]
        sV, sEV = Lar.skel_merge(sV, sEV, tmpV, tmpEV)
    
    end
    return sV,sEV
    
    end
        
        
    


sigma (generic function with 1 method)

In [151]:
#function arrsigma(sV,sEV,sigmavs,M)
function arrsigma(sV::Array,sEV::Lar.ChainOp,sigmavs::Array,M::Array)
    sV = sV[:, 1:2]
   
   
    nV, nEV, nFE = Lar.Arrangement.planar_arrangement(sV, sEV, sparsevec(ones(Int8, length(sigmavs))))
    if nV == nothing ## not possible !! ... (each original face maps to its decomposition)
        return [], spzeros(Int8, 0,0), spzeros(Int8, 0,0)
    end
    nvsize = size(nV, 1)
    nV = [nV zeros(nvsize) ones(nvsize)]*inv(M)[:, 1:3] ## ????
    
    return nV,nEV,nFE
end

arrsigma (generic function with 1 method)

In [157]:
using Base.Threads
function frag_faceOTT(V, EV, FE, sp_idx, sigma)
#function frag_faceOTT(V::Lar.Points, EV::Lar.ChainOp, FE::Lar.ChainOp, sp_idx::Array, sigma::Int)
    
    
    vs_num = size(V, 1)
    tV,sV,sEV,M,sigmavs = duetrasf(V,EV,FE,sigma,vs_num)
    count=0
    
    #sV,sEV= sigma(tV,EV,FE,sV,sEV,sp_idx,sigma)
    Threads.@threads for i in sp_idx[sigma]
         @show EV
         @show FE
         
         @inbounds tmpV, tmpEV = Lar.Arrangement.face_int(tV, EV, FE[i, :])
         
         
         @inbounds sV, sEV = Lar.skel_merge(sV, sEV, tmpV, tmpEV)
      
     end

    # computation of 2D arrangement of sigma face
    nV,nEV,nFE = arrsigma(sV,sEV,sigmavs,M)
    #sV = sV[:, 1:2]
    #nV, nEV, nFE = Lar.Arrangement.planar_arrangement(sV, sEV, sparsevec(ones(Int8, length(sigmavs))))
    #if nV == nothing ## not possible !! ... (each original face maps to its decomposition)
        #return [], spzeros(Int8, 0,0), spzeros(Int8, 0,0)
    #end
    #nvsize = size(nV, 1)
    #nV = [nV zeros(nvsize) ones(nvsize)]*inv(M)[:, 1:3] ## ????
    #print(typeof((vs_num,sV,sEV)))
    return nV, nEV, nFE

end


frag_faceOTT (generic function with 1 method)

In [158]:
using SparseArrays
using LinearAlgebraicRepresentation

Lar=LinearAlgebraicRepresentation



V = Float64[
        0 0 0; 0 1 0;
        1 1 0; 1 0 0;
        0 0 1; 0 1 1;
        1 1 1; 1 0 1
    ]

    EV = sparse(Int8[
        -1  1  0  0  0  0  0  0;
        0 -1  1  0  0  0  0  0;
        0  0 -1  1  0  0  0  0;
        -1  0  0  1  0  0  0  0;
        -1  0  0  0  1  0  0  0;
        0 -1  0  0  0  1  0  0;
        0  0 -1  0  0  0  1  0;
        0  0  0 -1  0  0  0  1;
        0  0  0  0 -1  1  0  0;
        0  0  0  0  0 -1  1  0;
        0  0  0  0  0  0 -1  1;
        0  0  0  0 -1  0  0  1;
    ])

    FE = sparse(Int8[
        1  1  1 -1  0  0  0  0  0  0  0  0;
        0  0  0  0  0  0  0  0 -1 -1 -1  1;
        -1  0  0  0  1 -1  0  0  1  0  0  0;
        0 -1  0  0  0  1 -1  0  0  1  0  0;
        0  0 -1  0  0  0  1 -1  0  0  1  0;
        0  0  0  1 -1  0  0  1  0  0  0 -1;
    ])
@time frag_faceOTT(Lar.Points(V),EV,FE,[2,4,5,7,8,9,10,11,10,50,11,2,3,4,5,6,1,2,35,6,7,4,],3)


EV = 
  [1 , 1]  =  -1
  [4 , 1]  =  -1
  [5 , 1]  =  -1
  [1 , 2]  =  1
  [2 , 2]  =  -1
  [6 , 2]  =  -1
  [2 , 3]  =  1
  [3 , 3]  =  -1
  [7 , 3]  =  -1
  [3 , 4]  =  1
  [4 , 4]  =  1
  [8 , 4]  =  -1
  [5 , 5]  =  1
  [9 , 5]  =  -1
  [12, 5]  =  -1
  [6 , 6]  =  1
  [9 , 6]  =  1
  [10, 6]  =  -1
  [7 , 7]  =  1
  [10, 7]  =  1
  [11, 7]  =  -1
  [8 , 8]  =  1
  [11, 8]  =  1
  [12, 8]  =  1
FE = 
  [1,  1]  =  1
  [3,  1]  =  -1
  [1,  2]  =  1
  [4,  2]  =  -1
  [1,  3]  =  1
  [5,  3]  =  -1
  [1,  4]  =  -1
  [6,  4]  =  1
  [3,  5]  =  1
  [6,  5]  =  -1
  [3,  6]  =  -1
  [4,  6]  =  1
  [4,  7]  =  -1
  [5,  7]  =  1
  [5,  8]  =  -1
  [6,  8]  =  1
  [2,  9]  =  -1
  [3,  9]  =  1
  [2, 10]  =  -1
  [4, 10]  =  1
  [2, 11]  =  -1
  [5, 11]  =  1
  [2, 12]  =  1
  [6, 12]  =  -1
  0.046627 seconds (87.69 k allocations: 4.714 MiB)


([0.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0; 0.0 1.0 1.0], 
  [1, 1]  =  -1
  [2, 1]  =  -1
  [1, 2]  =  1
  [3, 2]  =  -1
  [2, 3]  =  1
  [4, 3]  =  -1
  [3, 4]  =  1
  [4, 4]  =  1, 
  [1, 1]  =  1
  [1, 2]  =  -1
  [1, 3]  =  1
  [1, 4]  =  -1)

In [154]:
using BenchmarkTools

@btime frag_faceOTT(Lar.Points(V),EV,FE,[2,4,5],3)

EV = 
  [1 , 1]  =  -1
  [4 , 1]  =  -1
  [5 , 1]  =  -1
  [1 , 2]  =  1
  [2 , 2]  =  -1
  [6 , 2]  =  -1
  [2 , 3]  =  1
  [3 , 3]  =  -1
  [7 , 3]  =  -1
  [3 , 4]  =  1
  [4 , 4]  =  1
  [8 , 4]  =  -1
  [5 , 5]  =  1
  [9 , 5]  =  -1
  [12, 5]  =  -1
  [6 , 6]  =  1
  [9 , 6]  =  1
  [10, 6]  =  -1
  [7 , 7]  =  1
  [10, 7]  =  1
  [11, 7]  =  -1
  [8 , 8]  =  1
  [11, 8]  =  1
  [12, 8]  =  1
EV = 
  [1 , 1]  =  -1
  [4 , 1]  =  -1
  [5 , 1]  =  -1
  [1 , 2]  =  1
  [2 , 2]  =  -1
  [6 , 2]  =  -1
  [2 , 3]  =  1
  [3 , 3]  =  -1
  [7 , 3]  =  -1
  [3 , 4]  =  1
  [4 , 4]  =  1
  [8 , 4]  =  -1
  [5 , 5]  =  1
  [9 , 5]  =  -1
  [12, 5]  =  -1
  [6 , 6]  =  1
  [9 , 6]  =  1
  [10, 6]  =  -1
  [7 , 7]  =  1
  [10, 7]  =  1
  [11, 7]  =  -1
  [8 , 8]  =  1
  [11, 8]  =  1
  [12, 8]  =  1
EV = 
  [1 , 1]  =  -1
  [4 , 1]  =  -1
  [5 , 1]  =  -1
  [1 , 2]  =  1
  [2 , 2]  =  -1
  [6 , 2]  =  -1
  [2 , 3]  =  1
  [3 , 3]  =  -1
  [7 , 3]  =  -1
  [3 , 4]  =  1
  [4 , 4]  =  1
  [8 , 

EV = 
  [1 , 1]  =  -1
  [4 , 1]  =  -1
  [5 , 1]  =  -1
  [1 , 2]  =  1
  [2 , 2]  =  -1
  [6 , 2]  =  -1
  [2 , 3]  =  1
  [3 , 3]  =  -1
  [7 , 3]  =  -1
  [3 , 4]  =  1
  [4 , 4]  =  1
  [8 , 4]  =  -1
  [5 , 5]  =  1
  [9 , 5]  =  -1
  [12, 5]  =  -1
  [6 , 6]  =  1
  [9 , 6]  =  1
  [10, 6]  =  -1
  [7 , 7]  =  1
  [10, 7]  =  1
  [11, 7]  =  -1
  [8 , 8]  =  1
  [11, 8]  =  1
  [12, 8]  =  1
EV = 
  [1 , 1]  =  -1
  [4 , 1]  =  -1
  [5 , 1]  =  -1
  [1 , 2]  =  1
  [2 , 2]  =  -1
  [6 , 2]  =  -1
  [2 , 3]  =  1
  [3 , 3]  =  -1
  [7 , 3]  =  -1
  [3 , 4]  =  1
  [4 , 4]  =  1
  [8 , 4]  =  -1
  [5 , 5]  =  1
  [9 , 5]  =  -1
  [12, 5]  =  -1
  [6 , 6]  =  1
  [9 , 6]  =  1
  [10, 6]  =  -1
  [7 , 7]  =  1
  [10, 7]  =  1
  [11, 7]  =  -1
  [8 , 8]  =  1
  [11, 8]  =  1
  [12, 8]  =  1
EV = 
  [1 , 1]  =  -1
  [4 , 1]  =  -1
  [5 , 1]  =  -1
  [1 , 2]  =  1
  [2 , 2]  =  -1
  [6 , 2]  =  -1
  [2 , 3]  =  1
  [3 , 3]  =  -1
  [7 , 3]  =  -1
  [3 , 4]  =  1
  [4 , 4]  =  1
  [8 , 

EV = 
  [1 , 1]  =  -1
  [4 , 1]  =  -1
  [5 , 1]  =  -1
  [1 , 2]  =  1
  [2 , 2]  =  -1
  [6 , 2]  =  -1
  [2 , 3]  =  1
  [3 , 3]  =  -1
  [7 , 3]  =  -1
  [3 , 4]  =  1
  [4 , 4]  =  1
  [8 , 4]  =  -1
  [5 , 5]  =  1
  [9 , 5]  =  -1
  [12, 5]  =  -1
  [6 , 6]  =  1
  [9 , 6]  =  1
  [10, 6]  =  -1
  [7 , 7]  =  1
  [10, 7]  =  1
  [11, 7]  =  -1
  [8 , 8]  =  1
  [11, 8]  =  1
  [12, 8]  =  1
EV = 
  [1 , 1]  =  -1
  [4 , 1]  =  -1
  [5 , 1]  =  -1
  [1 , 2]  =  1
  [2 , 2]  =  -1
  [6 , 2]  =  -1
  [2 , 3]  =  1
  [3 , 3]  =  -1
  [7 , 3]  =  -1
  [3 , 4]  =  1
  [4 , 4]  =  1
  [8 , 4]  =  -1
  [5 , 5]  =  1
  [9 , 5]  =  -1
  [12, 5]  =  -1
  [6 , 6]  =  1
  [9 , 6]  =  1
  [10, 6]  =  -1
  [7 , 7]  =  1
  [10, 7]  =  1
  [11, 7]  =  -1
  [8 , 8]  =  1
  [11, 8]  =  1
  [12, 8]  =  1
EV = 
  [1 , 1]  =  -1
  [4 , 1]  =  -1
  [5 , 1]  =  -1
  [1 , 2]  =  1
  [2 , 2]  =  -1
  [6 , 2]  =  -1
  [2 , 3]  =  1
  [3 , 3]  =  -1
  [7 , 3]  =  -1
  [3 , 4]  =  1
  [4 , 4]  =  1
  [8 , 

EV = 
  [1 , 1]  =  -1
  [4 , 1]  =  -1
  [5 , 1]  =  -1
  [1 , 2]  =  1
  [2 , 2]  =  -1
  [6 , 2]  =  -1
  [2 , 3]  =  1
  [3 , 3]  =  -1
  [7 , 3]  =  -1
  [3 , 4]  =  1
  [4 , 4]  =  1
  [8 , 4]  =  -1
  [5 , 5]  =  1
  [9 , 5]  =  -1
  [12, 5]  =  -1
  [6 , 6]  =  1
  [9 , 6]  =  1
  [10, 6]  =  -1
  [7 , 7]  =  1
  [10, 7]  =  1
  [11, 7]  =  -1
  [8 , 8]  =  1
  [11, 8]  =  1
  [12, 8]  =  1
EV = 
  [1 , 1]  =  -1
  [4 , 1]  =  -1
  [5 , 1]  =  -1
  [1 , 2]  =  1
  [2 , 2]  =  -1
  [6 , 2]  =  -1
  [2 , 3]  =  1
  [3 , 3]  =  -1
  [7 , 3]  =  -1
  [3 , 4]  =  1
  [4 , 4]  =  1
  [8 , 4]  =  -1
  [5 , 5]  =  1
  [9 , 5]  =  -1
  [12, 5]  =  -1
  [6 , 6]  =  1
  [9 , 6]  =  1
  [10, 6]  =  -1
  [7 , 7]  =  1
  [10, 7]  =  1
  [11, 7]  =  -1
  [8 , 8]  =  1
  [11, 8]  =  1
  [12, 8]  =  1
EV = 
  [1 , 1]  =  -1
  [4 , 1]  =  -1
  [5 , 1]  =  -1
  [1 , 2]  =  1
  [2 , 2]  =  -1
  [6 , 2]  =  -1
  [2 , 3]  =  1
  [3 , 3]  =  -1
  [7 , 3]  =  -1
  [3 , 4]  =  1
  [4 , 4]  =  1
  [8 , 

  [12, 8]  =  1
EV = 
  [1 , 1]  =  -1
  [4 , 1]  =  -1
  [5 , 1]  =  -1
  [1 , 2]  =  1
  [2 , 2]  =  -1
  [6 , 2]  =  -1
  [2 , 3]  =  1
  [3 , 3]  =  -1
  [7 , 3]  =  -1
  [3 , 4]  =  1
  [4 , 4]  =  1
  [8 , 4]  =  -1
  [5 , 5]  =  1
  [9 , 5]  =  -1
  [12, 5]  =  -1
  [6 , 6]  =  1
  [9 , 6]  =  1
  [10, 6]  =  -1
  [7 , 7]  =  1
  [10, 7]  =  1
  [11, 7]  =  -1
  [8 , 8]  =  1
  [11, 8]  =  1
  [12, 8]  =  1
EV = 
  [1 , 1]  =  -1
  [4 , 1]  =  -1
  [5 , 1]  =  -1
  [1 , 2]  =  1
  [2 , 2]  =  -1
  [6 , 2]  =  -1
  [2 , 3]  =  1
  [3 , 3]  =  -1
  [7 , 3]  =  -1
  [3 , 4]  =  1
  [4 , 4]  =  1
  [8 , 4]  =  -1
  [5 , 5]  =  1
  [9 , 5]  =  -1
  [12, 5]  =  -1
  [6 , 6]  =  1
  [9 , 6]  =  1
  [10, 6]  =  -1
  [7 , 7]  =  1
  [10, 7]  =  1
  [11, 7]  =  -1
  [8 , 8]  =  1
  [11, 8]  =  1
  [12, 8]  =  1
EV = 
  [1 , 1]  =  -1
  [4 , 1]  =  -1
  [5 , 1]  =  -1
  [1 , 2]  =  1
  [2 , 2]  =  -1
  [6 , 2]  =  -1
  [2 , 3]  =  1
  [3 , 3]  =  -1
  [7 , 3]  =  -1
  [3 , 4]  =  1
  [4 , 

...

  [12, 8]  =  1
EV = 
  [1 , 1]  =  -1
  [4 , 1]  =  -1
  [5 , 1]  =  -1
  [1 , 2]  =  1
  [2 , 2]  =  -1
  [6 , 2]  =  -1
  [2 , 3]  =  1
  [3 , 3]  =  -1
  [7 , 3]  =  -1
  [3 , 4]  =  1
  [4 , 4]  =  1
  [8 , 4]  =  -1
  [5 , 5]  =  1
  [9 , 5]  =  -1
  [12, 5]  =  -1
  [6 , 6]  =  1
  [9 , 6]  =  1
  [10, 6]  =  -1
  [7 , 7]  =  1
  [10, 7]  =  1
  [11, 7]  =  -1
  [8 , 8]  =  1
  [11, 8]  =  1
  [12, 8]  =  1
EV = 
  [1 , 1]  =  -1
  [4 , 1]  =  -1
  [5 , 1]  =  -1
  [1 , 2]  =  1
  [2 , 2]  =  -1
  [6 , 2]  =  -1
  [2 , 3]  =  1
  [3 , 3]  =  -1
  [7 , 3]  =  -1
  [3 , 4]  =  1
  [4 , 4]  =  1
  [8 , 4]  =  -1
  [5 , 5]  =  1
  [9 , 5]  =  -1
  [12, 5]  =  -1
  [6 , 6]  =  1
  [9 , 6]  =  1
  [10, 6]  =  -1
  [7 , 7]  =  1
  [10, 7]  =  1
  [11, 7]  =  -1
  [8 , 8]  =  1
  [11, 8]  =  1
  [12, 8]  =  1
EV = 
  [1 , 1]  =  -1
  [4 , 1]  =  -1
  [5 , 1]  =  -1
  [1 , 2]  =  1
  [2 , 2]  =  -1
  [6 , 2]  =  -1
  [2 , 3]  =  1
  [3 , 3]  =  -1
  [7 , 3]  =  -1
  [3 , 4]  =  1
  [4 , 

  [12, 8]  =  1
EV = 
  [1 , 1]  =  -1
  [4 , 1]  =  -1
  [5 , 1]  =  -1
  [1 , 2]  =  1
  [2 , 2]  =  -1
  [6 , 2]  =  -1
  [2 , 3]  =  1
  [3 , 3]  =  -1
  [7 , 3]  =  -1
  [3 , 4]  =  1
  [4 , 4]  =  1
  [8 , 4]  =  -1
  [5 , 5]  =  1
  [9 , 5]  =  -1
  [12, 5]  =  -1
  [6 , 6]  =  1
  [9 , 6]  =  1
  [10, 6]  =  -1
  [7 , 7]  =  1
  [10, 7]  =  1
  [11, 7]  =  -1
  [8 , 8]  =  1
  [11, 8]  =  1
  [12, 8]  =  1
EV = 
  [1 , 1]  =  -1
  [4 , 1]  =  -1
  [5 , 1]  =  -1
  [1 , 2]  =  1
  [2 , 2]  =  -1
  [6 , 2]  =  -1
  [2 , 3]  =  1
  [3 , 3]  =  -1
  [7 , 3]  =  -1
  [3 , 4]  =  1
  [4 , 4]  =  1
  [8 , 4]  =  -1
  [5 , 5]  =  1
  [9 , 5]  =  -1
  [12, 5]  =  -1
  [6 , 6]  =  1
  [9 , 6]  =  1
  [10, 6]  =  -1
  [7 , 7]  =  1
  [10, 7]  =  1
  [11, 7]  =  -1
  [8 , 8]  =  1
  [11, 8]  =  1
  [12, 8]  =  1
EV = 
  [1 , 1]  =  -1
  [4 , 1]  =  -1
  [5 , 1]  =  -1
  [1 , 2]  =  1
  [2 , 2]  =  -1
  [6 , 2]  =  -1
  [2 , 3]  =  1
  [3 , 3]  =  -1
  [7 , 3]  =  -1
  [3 , 4]  =  1
  [4 , 

Excessive output truncated after 524610 bytes.

EV = 
  [1 , 1]  =  -1
  [4 , 1]  =  -1
  [5 , 1]  =  -1
  [1 , 2]  =  1
  [2 , 2]  =  -1
  [6 , 2]  =  -1
  [2 , 3]  =  1
  [3 , 3]  =  -1
  [7 , 3]  =  -1
  [3 , 4]  =  1
  [4 , 4]  =  1
  [8 , 4]  =  -1
  [5 , 5]  =  1
  [9 , 5]  =  -1
  [12, 5]  =  -1
  [6 , 6]  =  1
  [9 , 6]  =  1
  [10, 6]  =  -1
  [7 , 7]  =  1
  [10, 7]  =  1
  [11, 7]  =  -1
  [8 , 8]  =  1
  [11, 8]  =  1
  [12, 8]  =  1
EV = 
  [1 , 1]  =  -1
  [4 , 1]  =  -1
  [5 , 1]  =  -1
  [1 , 2]  =  1
  [2 , 2]  =  -1
  [6 , 2]  =  -1
  [2 , 3]  =  1
  [3 , 3]  =  -1
  [7 , 3]  =  -1
  [3 , 4]  =  1
  [4 , 4]  =  1
  [8 , 4]  =  -1
  [5 , 5]  =  1
  [9 , 5]  =  -1
  [12, 5]  =  -1
  [6 , 6]  =  1
  [9 , 6]  =  1
  [10, 6]  =  -1
  [7 , 7]  =  1
  [10, 7]  =  1
  [11, 7]  =  -1
  [8 , 8]  =  1
  [11, 8]  =  1
  [12, 8]  =  1
EV = 
  [1 , 1]  =  -1
  [4 , 1]  =  -1
  [5 , 1]  =  -1
  [1 , 2]  =  1
  [2 , 2]  =  -1
  [6 , 2]  =  -1
  [2 , 3]  =  1
  [3 , 3]  =  -1
  [7 , 3]  =  -1
  [3 , 4]  =  1
  [4 , 4]  =  1
  [8 , 

([0.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0; 0.0 1.0 1.0], 
  [1, 1]  =  -1
  [2, 1]  =  -1
  [1, 2]  =  1
  [3, 2]  =  -1
  [2, 3]  =  1
  [4, 3]  =  -1
  [3, 4]  =  1
  [4, 4]  =  1, 
  [1, 1]  =  1
  [1, 2]  =  -1
  [1, 3]  =  1
  [1, 4]  =  -1)